In [2]:
import csv

with open('./USA_Housing.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            print(f'\t{row[0]}.')
            line_count += 1
    print(f'Processed {line_count} lines.')

FileNotFoundError: [Errno 2] No such file or directory: 'USA_Housing.csv'